In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'gini', 'max_depth': 4}
----------
[[ 1362  3812]
 [    0 13996]]
----------
              precision    recall  f1-score   support

           0       1.00      0.26      0.42      5174
           1       0.79      1.00      0.88     13996

    accuracy                           0.80     19170
   macro avg       0.89      0.63      0.65     19170
weighted avg       0.84      0.80      0.76     19170

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.15508461, 0.19547665, 0.22290409, 0.27277148, 0.30019796,
        0.30518329, 0.35505068, 0.39843488, 0.46974432, 0.46774948,
        0.48969018, 0.48919272, 0.51512146, 0.54154956, 0.57396376,
        0.21991122, 0.26030374, 0.293715  , 0.37699139, 0.38945866,
        0.40292203, 0.45777607, 0.53207672, 0.58842659, 0.59840059,
        0.79487455, 0.84424233, 0.81880891, 0.78589845, 0.7315762 ]),
 'std_fit_time': array([0.00049925, 0.01496017, 0.01545918, 0.02144158, 0.03590262,
        0.01396191, 0.01495969, 0.01246786, 0.01795161, 0.01695478,
        0.00498736, 0.01845121, 0.0054853 , 0.00299013, 0.01845086,
        0.00049841, 0.01097035, 0.00448918, 0.01695526, 0.01645541,
        0.00698125, 0.02493405, 0.00349081, 0.04288554, 0.00997353,
        0.06382859, 0.00548553, 0.01196849, 0.00299191, 0.03939462]),
 'mean_score_time': array([0.00997317, 0.01296568, 0.01146865, 0.01146722, 0.01346362,
        0.0179522 , 0.01246738, 0.01346409, 0.01246715, 0.01